# Amazon SageMaker Autopilot Candidate Definition Notebook

This notebook was automatically generated by the AutoML job **ArvatoExperiment**.
This notebook allows you to customize the candidate definitions and execute the SageMaker Autopilot workflow.

The dataset has **368** columns and the column named **RESPONSE** is used as
the target column. This is being treated as a **BinaryClassification** problem. The dataset also has **2** classes.
This notebook will build a **[BinaryClassification](https://en.wikipedia.org/wiki/Binary_classification)** model that
**maximizes** the "**F1**" quality metric of the trained models.
The "**F1**" metric applies for binary classification with a positive and negative class. It mixes between precision and recall, and is recommended in cases where there are more negative examples compared to positive examples.

As part of the AutoML job, the input dataset has been randomly split into two pieces, one for **training** and one for
**validation**. This notebook helps you inspect and modify the data transformation approaches proposed by Amazon SageMaker Autopilot. You can interactively
train the data transformation models and use them to transform the data. Finally, you can execute a multiple algorithm hyperparameter optimization (multi-algo HPO)
job that helps you find the best model for your dataset by jointly optimizing the data transformations and machine learning algorithms.

<div class="alert alert-info"> 💡 <strong> Available Knobs</strong>
Look for sections like this for recommended settings that you can change.
</div>


---

## Contents

1. [Sagemaker Setup](#Sagemaker-Setup)
    1. [Downloading Generated Candidates](#Downloading-Generated-Modules)
    1. [SageMaker Autopilot Job and Amazon Simple Storage Service (Amazon S3) Configuration](#SageMaker-Autopilot-Job-and-Amazon-Simple-Storage-Service-(Amazon-S3)-Configuration)
1. [Candidate Pipelines](#Candidate-Pipelines)
    1. [Generated Candidates](#Generated-Candidates)
    1. [Selected Candidates](#Selected-Candidates)
1. [Executing the Candidate Pipelines](#Executing-the-Candidate-Pipelines)
    1. [Run Data Transformation Steps](#Run-Data-Transformation-Steps)
    1. [Multi Algorithm Hyperparameter Tuning](#Multi-Algorithm-Hyperparameter-Tuning)
1. [Model Selection and Deployment](#Model-Selection-and-Deployment)
    1. [Tuning Job Result Overview](#Tuning-Job-Result-Overview)
    1. [Model Deployment](#Model-Deployment)

---

## Sagemaker Setup

Before you launch the SageMaker Autopilot jobs, we'll setup the environment for Amazon SageMaker
- Check environment & dependencies.
- Create a few helper objects/function to organize input/output data and SageMaker sessions.

**Minimal Environment Requirements**

- Jupyter: Tested on `JupyterLab 1.0.6`, `jupyter_core 4.5.0` and `IPython 6.4.0`
- Kernel: `conda_python3`
- Dependencies required
  - `sagemaker-python-sdk>=v1.43.4`
- Expected Execution Role/permission
  - S3 access to the bucket that stores the notebook.

### Downloading Generated Modules
Download the generated data transformation modules and an SageMaker Autopilot helper module used by this notebook.
Those artifacts will be downloaded to **ArvatoExperiment-artifacts** folder.

In [1]:
!mkdir -p ArvatoExperiment-artifacts
!aws s3 sync s3://sagemaker-studio-848439228145-ktemu68nnu/ArvatoExperiment/sagemaker-automl-candidates/pr-1-d149676d02b64c93aca959e42e072009739d8d9b21d44aeb835d9e9f9b/generated_module ArvatoExperiment-artifacts/generated_module --only-show-errors
!aws s3 sync s3://sagemaker-studio-848439228145-ktemu68nnu/ArvatoExperiment/sagemaker-automl-candidates/pr-1-d149676d02b64c93aca959e42e072009739d8d9b21d44aeb835d9e9f9b/notebooks/sagemaker_automl ArvatoExperiment-artifacts/sagemaker_automl --only-show-errors

import sys
sys.path.append("ArvatoExperiment-artifacts")

### SageMaker Autopilot Job and Amazon Simple Storage Service (Amazon S3) Configuration

The following configuration has been derived from the SageMaker Autopilot job. These items configure where this notebook will
look for generated candidates, and where input and output data is stored on Amazon S3.

In [2]:
from sagemaker_automl import uid, AutoMLLocalRunConfig

# Where the preprocessed data from the existing AutoML job is stored
BASE_AUTOML_JOB_NAME = 'ArvatoExperiment'
BASE_AUTOML_JOB_CONFIG = {
    'automl_job_name': BASE_AUTOML_JOB_NAME,
    'automl_output_s3_base_path': 's3://sagemaker-studio-848439228145-ktemu68nnu/ArvatoExperiment',
    'data_transformer_image_repo_version': '0.1.0-cpu-py3',
    'algo_image_repo_versions': {'xgboost': '0.90-1-cpu-py3', 'linear-learner': 'latest'}
}

# Path conventions of the output data storage path from the local AutoML job run of this notebook
LOCAL_AUTOML_JOB_NAME = 'ArvatoExpe-notebook-run-{}'.format(uid())
LOCAL_AUTOML_JOB_CONFIG = {
    'local_automl_job_name': LOCAL_AUTOML_JOB_NAME,
    'local_automl_job_output_s3_base_path': 's3://sagemaker-studio-848439228145-ktemu68nnu/ArvatoExperiment/{}'.format(LOCAL_AUTOML_JOB_NAME),
    'data_processing_model_dir': 'data-processor-models',
    'data_processing_transformed_output_dir': 'transformed-data',
    'multi_algo_tuning_output_dir': 'multi-algo-tuning'
}

AUTOML_LOCAL_RUN_CONFIG = AutoMLLocalRunConfig(
    role='arn:aws:iam::848439228145:role/service-role/AmazonSageMaker-ExecutionRole-20200405T201559',
    base_automl_job_config=BASE_AUTOML_JOB_CONFIG,
    local_automl_job_config=LOCAL_AUTOML_JOB_CONFIG,
    security_config={'EnableInterContainerTrafficEncryption': False, 'VpcConfig': {}})

AUTOML_LOCAL_RUN_CONFIG.display()

This notebook is initialized to use the following configuration: 
        <table>
        <tr><th colspan=2>Name</th><th>Value</th></tr>
        <tr><th>General</th><th>Role</th><td>arn:aws:iam::848439228145:role/service-role/AmazonSageMaker-ExecutionRole-20200405T201559</td></tr>
        <tr><th rowspan=2>Base AutoML Job</th><th>Job Name</th><td>ArvatoExperiment</td></tr>
        <tr><th>Base Output S3 Path</th><td>s3://sagemaker-studio-848439228145-ktemu68nnu/ArvatoExperiment</td></tr>
        <tr><th rowspan=5>Interactive Job</th><th>Job Name</th><td>ArvatoExpe-notebook-run-23-08-53-56</td></tr>
        <tr><th>Base Output S3 Path</th><td>s3://sagemaker-studio-848439228145-ktemu68nnu/ArvatoExperiment/ArvatoExpe-notebook-run-23-08-53-56</td></tr>
        <tr><th>Data Processing Trained Model Directory</th><td>s3://sagemaker-studio-848439228145-ktemu68nnu/ArvatoExperiment/ArvatoExpe-notebook-run-23-08-53-56/data-processor-models</td></tr>
        <tr><th>Data Processing Transformed Output</th><td>s3://sagemaker-studio-848439228145-ktemu68nnu/ArvatoExperiment/ArvatoExpe-notebook-run-23-08-53-56/transformed-data</td></tr>
        <tr><th>Algo Tuning Model Output Directory</th><td>s3://sagemaker-studio-848439228145-ktemu68nnu/ArvatoExperiment/ArvatoExpe-notebook-run-23-08-53-56/multi-algo-tuning</td></tr>
        </table>
        

## Candidate Pipelines

The `AutoMLLocalRunner` keeps track of selected candidates and automates many of the steps needed to execute feature engineering and tuning steps.

In [3]:
from sagemaker_automl import AutoMLInteractiveRunner, AutoMLLocalCandidate

automl_interactive_runner = AutoMLInteractiveRunner(AUTOML_LOCAL_RUN_CONFIG)

### Generated Candidates

The SageMaker Autopilot Job has analyzed the dataset and has generated **10** machine learning
pipeline(s) that use **2** algorithm(s). Each pipeline contains a set of feature transformers and an
algorithm.

<div class="alert alert-info"> 💡 <strong> Available Knobs</strong>

1. The resource configuration: instance type & count
1. Select candidate pipeline definitions by cells
1. The linked data transformation script can be reviewed and updated. Please refer to the [README.md](./ArvatoExperiment-artifacts/generated_module/README.md) for detailed customization instructions.
</div>

**[dpp0-xgboost](ArvatoExperiment-artifacts/generated_module/candidate_data_processors/dpp0.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [5]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp0",
        "training_resource_config": {
            "instance_type": "ml.m5.large",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.large",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "application/x-recordio-protobuf",
        "sparse_encoding": True
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.large",
            "instance_count": 1,
        }
    }
})

2020-05-23 08:54:49,422 INFO root: Warning: pipeline candidate dpp0-xgboost has already been selected, replacing


**[dpp1-linear-learner](ArvatoExperiment-artifacts/generated_module/candidate_data_processors/dpp1.py)**: This data transformation strategy first transforms 'numeric' features using [combined RobustImputer and RobustMissingIndicator](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py) followed by [LogExtremeValuesTransformer](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustPCA](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/decomposition/robust_pca.py) followed by [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *linear-learner* model. Here is the definition:

In [6]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp1",
        "training_resource_config": {
            "instance_type": "ml.m5.large",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.large",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "application/x-recordio-protobuf",
        "sparse_encoding": False
    },
    "algorithm": {
        "name": "linear-learner",
        "training_resource_config": {
            "instance_type": "ml.m5.large",
            "instance_count": 1,
        }
    }
})

**[dpp2-xgboost](ArvatoExperiment-artifacts/generated_module/candidate_data_processors/dpp2.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustPCA](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/decomposition/robust_pca.py) followed by [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [7]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp2",
        "training_resource_config": {
            "instance_type": "ml.m5.large",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.large",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "text/csv",
        "sparse_encoding": False
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.large",
            "instance_count": 1,
        }
    }
})

**[dpp3-xgboost](ArvatoExperiment-artifacts/generated_module/candidate_data_processors/dpp3.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [8]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp3",
        "training_resource_config": {
            "instance_type": "ml.m5.large",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.large",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "application/x-recordio-protobuf",
        "sparse_encoding": True
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.large",
            "instance_count": 1,
        }
    }
})

**[dpp4-linear-learner](ArvatoExperiment-artifacts/generated_module/candidate_data_processors/dpp4.py)**: This data transformation strategy first transforms 'numeric' features using [combined RobustImputer and RobustMissingIndicator](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py) followed by [QuantileExtremeValuesTransformer](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustPCA](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/decomposition/robust_pca.py) followed by [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *linear-learner* model. Here is the definition:

In [9]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp4",
        "training_resource_config": {
            "instance_type": "ml.m5.large",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.large",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "application/x-recordio-protobuf",
        "sparse_encoding": False
    },
    "algorithm": {
        "name": "linear-learner",
        "training_resource_config": {
            "instance_type": "ml.m5.large",
            "instance_count": 1,
        }
    }
})

**[dpp5-xgboost](ArvatoExperiment-artifacts/generated_module/candidate_data_processors/dpp5.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py), 'text' features using [MultiColumnTfidfVectorizer](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [10]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp5",
        "training_resource_config": {
            "instance_type": "ml.m5.large",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.large",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "application/x-recordio-protobuf",
        "sparse_encoding": True
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.large",
            "instance_count": 1,
        }
    }
})

**[dpp6-xgboost](ArvatoExperiment-artifacts/generated_module/candidate_data_processors/dpp6.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustPCA](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/decomposition/robust_pca.py) followed by [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [11]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp6",
        "training_resource_config": {
            "instance_type": "ml.m5.large",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.large",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "text/csv",
        "sparse_encoding": False
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.large",
            "instance_count": 1,
        }
    }
})

**[dpp7-xgboost](ArvatoExperiment-artifacts/generated_module/candidate_data_processors/dpp7.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [12]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp7",
        "training_resource_config": {
            "instance_type": "ml.m5.large",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.large",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "application/x-recordio-protobuf",
        "sparse_encoding": True
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.large",
            "instance_count": 1,
        }
    }
})

**[dpp8-linear-learner](ArvatoExperiment-artifacts/generated_module/candidate_data_processors/dpp8.py)**: This data transformation strategy first transforms 'numeric' features using [combined RobustImputer and RobustMissingIndicator](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py) followed by [QuantileExtremeValuesTransformer](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustPCA](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/decomposition/robust_pca.py) followed by [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *linear-learner* model. Here is the definition:

In [13]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp8",
        "training_resource_config": {
            "instance_type": "ml.m5.large",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.large",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "application/x-recordio-protobuf",
        "sparse_encoding": False
    },
    "algorithm": {
        "name": "linear-learner",
        "training_resource_config": {
            "instance_type": "ml.m5.large",
            "instance_count": 1,
        }
    }
})

**[dpp9-xgboost](ArvatoExperiment-artifacts/generated_module/candidate_data_processors/dpp9.py)**: This data transformation strategy first transforms 'numeric' features using [combined RobustImputer and RobustMissingIndicator](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py) followed by [QuantileExtremeValuesTransformer](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustPCA](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/decomposition/robust_pca.py) followed by [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [14]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp9",
        "training_resource_config": {
            "instance_type": "ml.m5.large",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.large",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "text/csv",
        "sparse_encoding": False
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.large",
            "instance_count": 1,
        }
    }
})

### Selected Candidates

You have selected the following candidates (please run the cell below and click on the feature transformer links for details):

In [15]:
automl_interactive_runner.display_candidates()

Candidate Name,Algorithm,Feature Transformer
dpp0-xgboost,xgboost,dpp0.py
dpp1-linear-learner,linear-learner,dpp1.py
dpp2-xgboost,xgboost,dpp2.py
dpp3-xgboost,xgboost,dpp3.py
dpp4-linear-learner,linear-learner,dpp4.py
dpp5-xgboost,xgboost,dpp5.py
dpp6-xgboost,xgboost,dpp6.py
dpp7-xgboost,xgboost,dpp7.py
dpp8-linear-learner,linear-learner,dpp8.py
dpp9-xgboost,xgboost,dpp9.py


The feature engineering pipeline consists of two SageMaker jobs:

1. Generated trainable data transformer Python modules like [dpp0.py](ArvatoExperiment-artifacts/generated_module/candidate_data_processors/dpp0.py), which has been downloaded to local file system
2. A **training** job to train the data transformers
3. A **batch transform** job to apply the trained transformation to the dataset to generate the algorithm compatible data

The transformers and its training pipeline are built using open sourced **[sagemaker-scikit-learn-container][]** and **[sagemaker-scikit-learn-extension][]**.

[sagemaker-scikit-learn-container]: https://github.com/aws/sagemaker-scikit-learn-container
[sagemaker-scikit-learn-extension]: https://github.com/aws/sagemaker-scikit-learn-extension

## Executing the Candidate Pipelines

Each candidate pipeline consists of two steps, feature transformation and algorithm training.
For efficiency first execute the feature transformation step which will generate a featurized dataset on S3
for each pipeline.

After each featurized dataset is prepared, execute a multi-algorithm tuning job that will run tuning jobs
in parallel for each pipeline. This tuning job will execute training jobs to find the best set of
hyper-parameters for each pipeline, as well as finding the overall best performing pipeline.

### Run Data Transformation Steps

Now you are ready to start execution all data transformation steps.  The cell below may take some time to finish,
feel free to go grab a cup of coffee. To expedite the process you can set the number of `parallel_jobs` to be up to 10.
Please check the account limits to increase the limits before increasing the number of jobs to run in parallel.

In [16]:
automl_interactive_runner.fit_data_transformers(parallel_jobs=2)

2020-05-23 08:57:44,879 INFO root: [Worker_0:dpp0-xgboost]Executing step: train_data_transformer
2020-05-23 08:57:45,111 INFO sagemaker: Creating training-job with name: ArvatoExpe-notebook-run-23-08-53-56-dpp0-train-23-08-57-44

2020-05-23 08:57:45 Starting - Starting the training job
2020-05-23 08:57:47 Starting - Launching requested ML instances2020-05-23 08:57:52,891 INFO root: [Worker_1:dpp1-linear-learner]Executing step: train_data_transformer
2020-05-23 08:57:53,064 INFO sagemaker: Creating training-job with name: ArvatoExpe-notebook-run-23-08-53-56-dpp1-train-23-08-57-44

2020-05-23 08:57:53 Starting - Starting the training job.
2020-05-23 08:57:55 Starting - Launching requested ML instances......................
2020-05-23 08:58:53 Starting - Preparing the instances for training..
2020-05-23 08:58:59 Starting - Preparing the instances for training..............
2020-05-23 08:59:37 Downloading - Downloading input data
2020-05-23 08:59:40 Downloading - Downloading input data....

### Multi Algorithm Hyperparameter Tuning

Now that the algorithm compatible trasformed datasets are ready, you can start the multi-algorithm model tuning job
to find the best predictive model. The following algorithm training job configuration for each
algorithm is auto-generated by the AutoML Job as part of the recommendation.

<div class="alert alert-info"> 💡 <strong> Available Knobs</strong>

1. Hyperparameter ranges
2. Objective metrics
3. Recommended static algorithm hyperparameters.

Please refers to [Xgboost tuning](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost-tuning.html) and [Linear learner tuning](https://docs.aws.amazon.com/sagemaker/latest/dg/linear-learner-tuning.html) for detailed explanations of the parameters.
</div>

The AutoML recommendation job has recommended the following hyperparameters, objectives and accuracy metrics for
the algorithm and problem type:

In [17]:
ALGORITHM_OBJECTIVE_METRICS = {
    'xgboost': 'validation:f1',
    'linear-learner': 'validation:binary_f_beta',
}

STATIC_HYPERPARAMETERS = {
    'xgboost': {
        'objective': 'binary:hinge',
    },
    'linear-learner': {
        'predictor_type': 'binary_classifier',
        'loss': 'logistic',
        'mini_batch_size': 800,
        'binary_classifier_model_selection_criteria': 'loss_function',
        'num_models': 1,
    },
}

The following tunable hyperparameters search ranges are recommended for the Multi-Algo tuning job:

In [18]:
from sagemaker.parameter import CategoricalParameter, ContinuousParameter, IntegerParameter

ALGORITHM_TUNABLE_HYPERPARAMETER_RANGES = {
    'xgboost': {
        'num_round': IntegerParameter(2, 512, scaling_type='Logarithmic'),
        'max_depth': IntegerParameter(2, 32, scaling_type='Logarithmic'),
        'eta': ContinuousParameter(1e-3, 1.0, scaling_type='Logarithmic'),
        'gamma': ContinuousParameter(1e-6, 64.0, scaling_type='Logarithmic'),
        'min_child_weight': ContinuousParameter(1e-6, 32.0, scaling_type='Logarithmic'),
        'subsample': ContinuousParameter(0.5, 1.0, scaling_type='Linear'),
        'colsample_bytree': ContinuousParameter(0.3, 1.0, scaling_type='Linear'),
        'lambda': ContinuousParameter(1e-6, 2.0, scaling_type='Logarithmic'),
        'alpha': ContinuousParameter(1e-6, 2.0, scaling_type='Logarithmic'),
    },
    'linear-learner': {
        'wd': ContinuousParameter(1e-7, 1.0, scaling_type='Logarithmic'),
        'l1': ContinuousParameter(1e-7, 1.0, scaling_type='Logarithmic'),
        'learning_rate': ContinuousParameter(1e-5, 1.0, scaling_type='Logarithmic'),
        'positive_example_weight_mult': CategoricalParameter(['balanced', '0.01', '1', '100']),
    },
}

#### Prepare Multi-Algorithm Tuner Input

To use the multi-algorithm HPO tuner, prepare some inputs and parameters. Prepare a dictionary whose key is the name of the trained pipeline candidates and the values are respectively:

1. Estimators for the recommended algorithm
2. Hyperparameters search ranges
3. Objective metrics

In [19]:
multi_algo_tuning_parameters = automl_interactive_runner.prepare_multi_algo_parameters(
    objective_metrics=ALGORITHM_OBJECTIVE_METRICS,
    static_hyperparameters=STATIC_HYPERPARAMETERS,
    hyperparameters_search_ranges=ALGORITHM_TUNABLE_HYPERPARAMETER_RANGES)

Below you prepare the inputs data to the multi-algo tuner:

In [20]:
multi_algo_tuning_inputs = automl_interactive_runner.prepare_multi_algo_inputs()

#### Create Multi-Algorithm Tuner

With the recommended Hyperparameter ranges and the transformed dataset, create a multi-algorithm model tuning job
that coordinates hyper parameter optimizations across the different possible algorithms and feature processing strategies.

<div class="alert alert-info"> 💡 <strong> Available Knobs</strong>

1. Tuner strategy: [Bayesian](https://en.wikipedia.org/wiki/Hyperparameter_optimization#Bayesian_optimization), [Random Search](https://en.wikipedia.org/wiki/Hyperparameter_optimization#Random_search)
2. Objective type: `Minimize`, `Maximize`, see [optimization](https://en.wikipedia.org/wiki/Mathematical_optimization)
3. Max Job size: the max number of training jobs HPO would be launching to run experiments. Note the default value is **250**
    which is the default of the managed flow.
4. Parallelism. Number of jobs that will be executed in parallel. Higher value will expedite the tuning process.
    Please check the account limits to increase the limits before increasing the number of jobs to run in parallel
5. Please use a different tuning job name if you re-run this cell after applied customizations.
</div>

In [21]:
from sagemaker.tuner import HyperparameterTuner

base_tuning_job_name = "{}-tuning".format(AUTOML_LOCAL_RUN_CONFIG.local_automl_job_name)

tuner = HyperparameterTuner.create(
    base_tuning_job_name=base_tuning_job_name,
    strategy='Bayesian',
    objective_type='Maximize',
    max_parallel_jobs=2,
    max_jobs=250,
    **multi_algo_tuning_parameters,
)

#### Run Multi-Algorithm Tuning

Now you are ready to start running the **Multi-Algo Tuning** job. After the job is finished, store the tuning job name which you use to select models in the next section.
The tuning process will take some time, please track the progress in the Amazon SageMaker Hyperparameter tuning jobs console.

In [22]:
from IPython.display import display, Markdown

# Run tuning
tuner.fit(inputs=multi_algo_tuning_inputs, include_cls_metadata=None)
tuning_job_name = tuner.latest_tuning_job.name

display(
    Markdown(f"Tuning Job {tuning_job_name} started, please track the progress from [here](https://{AUTOML_LOCAL_RUN_CONFIG.region}.console.aws.amazon.com/sagemaker/home?region={AUTOML_LOCAL_RUN_CONFIG.region}#/hyper-tuning-jobs/{tuning_job_name})"))

# Wait for tuning job to finish
tuner.wait()

2020-05-23 09:44:25,239 INFO root: _TuningJob.start_new!!!
2020-05-23 09:44:25,244 INFO sagemaker: Creating hyperparameter tuning job with name: ArvatoExpe-notebook--200523-0944


Tuning Job ArvatoExpe-notebook--200523-0944 started, please track the progress from [here](https://eu-west-1.console.aws.amazon.com/sagemaker/home?region=eu-west-1#/hyper-tuning-jobs/ArvatoExpe-notebook--200523-0944)

...................................................................................................................................................................................................................................................................................................................................................................................................................................................._s


## Model Selection and Deployment

This section guides you through the model selection process. Afterward, you construct an inference pipeline
on Amazon SageMaker to host the best candidate.

Because you executed the feature transformation and algorithm training in two separate steps, you now need to manually
link each trained model with the feature transformer that it is associated with. When running a regular Amazon
SageMaker Autopilot job, this will automatically be done for you.

### Tuning Job Result Overview

The performance of each candidate pipeline can be viewed as a Pandas dataframe. For more interactive usage please
refers to [model tuning monitor](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-monitor.html).

In [23]:
from pprint import pprint
from sagemaker.analytics import HyperparameterTuningJobAnalytics

SAGEMAKER_SESSION = AUTOML_LOCAL_RUN_CONFIG.sagemaker_session
SAGEMAKER_ROLE = AUTOML_LOCAL_RUN_CONFIG.role

tuner_analytics = HyperparameterTuningJobAnalytics(
    tuner.latest_tuning_job.name, sagemaker_session=SAGEMAKER_SESSION)

df_tuning_job_analytics = tuner_analytics.dataframe()

# Sort the tuning job analytics by the final metrics value
df_tuning_job_analytics.sort_values(
    by=['FinalObjectiveValue'],
    inplace=True,
    ascending=False if tuner.objective_type == "Maximize" else True)

# Show detailed analytics for the top 20 models
df_tuning_job_analytics.head(20)

,alpha,colsample_bytree,eta,gamma,lambda,max_depth,min_child_weight,num_round,subsample,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds,TrainingJobDefinitionName,l1,learning_rate,positive_example_weight_mult,wd
13,0.010386,0.825286,0.207616,2.182224,0.000871,21.0,0.165931,8.0,0.988949,ArvatoExpe-notebook--200523-0944-003-dad76254,Completed,0.504488,2020-05-23 09:52:10+00:00,2020-05-23 09:53:33+00:00,83.0,dpp5-xgboost,NaN,NaN,NaN,NaN
3,0.000001,0.378623,0.294812,6.078858,0.007390,9.0,15.033048,26.0,0.606877,ArvatoExpe-notebook--200523-0944-013-e8c68a8d,Completed,0.496984,2020-05-23 10:16:21+00:00,2020-05-23 10:18:09+00:00,108.0,dpp2-xgboost,NaN,NaN,NaN,NaN
9,0.035363,0.824353,0.008977,0.000055,0.646468,2.0,3.991928,428.0,0.771490,ArvatoExpe-notebook--200523-0944-007-9712a953,Completed,0.496984,2020-05-23 10:03:12+00:00,2020-05-23 10:05:52+00:00,160.0,dpp3-xgboost,NaN,NaN,NaN,NaN
11,0.002272,0.513653,0.012166,0.000266,0.046147,2.0,2.495125,340.0,0.883905,ArvatoExpe-notebook--200523-0944-005-47f65142,Completed,0.496984,2020-05-23 09:55:30+00:00,2020-05-23 10:01:46+00:00,376.0,dpp9-xgboost,NaN,NaN,NaN,NaN
12,0.001873,0.539304,0.002351,2.815771,0.000003,3.0,0.000003,472.0,0.698045,ArvatoExpe-notebook--200523-0944-004-89e49eb8,Completed,0.496984,2020-05-23 09:54:06+00:00,2020-05-23 09:56:57+00:00,171.0,dpp7-xgboost,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ArvatoExpe-notebook--200523-0944-014-22cfbbe6,Completed,0.032986,2020-05-23 10:17:34+00:00,2020-05-23 10:18:44+00:00,70.0,dpp8-linear-learner,7.361511e-03,0.889374,balanced,0.001121
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ArvatoExpe-notebook--200523-0944-008-1cbfe273,Completed,0.029341,2020-05-23 10:04:32+00:00,2020-05-23 10:05:53+00:00,81.0,dpp1-linear-learner,4.193069e-07,0.180807,balanced,0.199269
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ArvatoExpe-notebook--200523-0944-006-3fdefd33,Completed,0.027517,2020-05-23 09:59:24+00:00,2020-05-23 10:00:40+00:00,76.0,dpp8-linear-learner,5.996115e-04,0.000024,100,0.048836
4,0.000038,0.448690,0.001083,0.006412,0.012774,9.0,0.776570,40.0,0.833810,ArvatoExpe-notebook--200523-0944-012-30650e54,Completed,0.011849,2020-05-23 10:12:23+00:00,2020-05-23 10:15:09+00:00,166.0,dpp6-xgboost,NaN,NaN,NaN,NaN
5,0.002988,0.530300,0.019854,2.106704,0.000514,2.0,0.000103,3.0,0.517566,ArvatoExpe-notebook--200523-0944-011-a1f9cfd6,Completed,0.011849,2020-05-23 10:12:02+00:00,2020-05-23 10:13:16+00:00,74.0,dpp5-xgboost,NaN,NaN,NaN,NaN


The best training job can be selected as below:

<div class="alert alert-info"> 💡 <strong>Tips: </strong>
You could select alternative job by using the value from `TrainingJobName` column above and assign to `best_training_job` below
</div>

In [24]:
attached_tuner = HyperparameterTuner.attach(tuner.latest_tuning_job.name, sagemaker_session=SAGEMAKER_SESSION)
best_training_job = attached_tuner.best_training_job()

print("Best Multi Algorithm HPO training job name is {}".format(best_training_job))

Best Multi Algorithm HPO training job name is ArvatoExpe-notebook--200523-0944-003-dad76254


### Linking Best Training Job with Feature Pipelines

Finally, deploy the best training job to Amazon SageMaker along with its companion feature engineering models.
At the end of the section, you get an endpoint that's ready to serve online inference or start batch transform jobs!

Deploy a [PipelineModel](https://sagemaker.readthedocs.io/en/stable/pipeline.html) that has multiple containers of the following:

1. Data Transformation Container: a container built from the model we selected and trained during the data transformer sections
2. Algorithm Container: a container built from the trained model we selected above from the best HPO training job.
3. Inverse Label Transformer Container: a container that converts numerical intermediate prediction value back to non-numerical label value.

Get both best data transformation model and algorithm model from best training job and create an pipeline model:

In [25]:
from sagemaker.estimator import Estimator
from sagemaker import PipelineModel

# Get a data transformation model from chosen candidate
best_candidate = automl_interactive_runner.choose_candidate(df_tuning_job_analytics, best_training_job)
best_data_transformer_model = best_candidate.get_data_transformer_model(role=SAGEMAKER_ROLE, sagemaker_session=SAGEMAKER_SESSION)

# Our first data transformation container will always return recordio-protobuf format
best_data_transformer_model.env["SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT"] = 'application/x-recordio-protobuf'
# Add environment variable for sparse encoding
if best_candidate.data_transformer_step.sparse_encoding:
    best_data_transformer_model.env["AUTOML_SPARSE_ENCODE_RECORDIO_PROTOBUF"] = '1'

# Get a algo model from chosen training job of the candidate
algo_estimator = Estimator.attach(best_training_job)
best_algo_model = algo_estimator.create_model(env={'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT':"text/csv"})

# Final pipeline model is composed of data transformation models and algo model and an
# inverse label transform model if we need to transform the intermediates back to non-numerical value
model_containers = [best_data_transformer_model, best_algo_model]
if best_candidate.transforms_label:
    model_containers.append(best_candidate.get_data_transformer_model(
        transform_mode="inverse-label-transform",
        role=SAGEMAKER_ROLE,
        sagemaker_session=SAGEMAKER_SESSION))

pipeline_model = PipelineModel(
    name="AutoML-{}".format(AUTOML_LOCAL_RUN_CONFIG.local_automl_job_name),
    role=SAGEMAKER_ROLE,
    models=model_containers,
    vpc_config=AUTOML_LOCAL_RUN_CONFIG.vpc_config)

2020-05-23 10:22:52,190 INFO root: Chosen Data Processing pipeline candidate name is dpp5-xgboost
2020-05-23 09:19:14 Starting - Preparing the instances for training
2020-05-23 09:19:14 Downloading - Downloading input data
2020-05-23 09:19:14 Training - Training image download completed. Training in progress.
2020-05-23 09:19:14 Uploading - Uploading generated training model
2020-05-23 09:19:14 Completed - Training job completed2020-05-23 09:18:12,150 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2020-05-23 09:18:12,153 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-05-23 09:18:12,162 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-05-23 09:18:12,410 sagemaker-containers INFO     Module default_user_module_name does not provide a setup.py. 
Generating setup.py
2020-05-23 09:18:12,410 sagemaker-containers INFO     Generating setup.cfg
2020-05-23 09:18:12,410 sagemaker-containers 

### Deploying Best Pipeline

<div class="alert alert-info"> 💡 <strong> Available Knobs</strong>

1. You can customize the initial instance count and instance type used to deploy this model.
2. Endpoint name can be changed to avoid conflict with existing endpoints.

</div>

Finally, deploy the model to SageMaker to make it functional.

In [28]:
pipeline_transformer = pipeline_model.transformer(instance_count=1,
                      instance_type='ml.m5.large')

2020-05-23 10:26:34,214 INFO sagemaker: Creating model with name: AutoML-ArvatoExpe-notebook-run-23-08-53-56


In [29]:
pipeline_transformer.transform(data='s3://sagemaker-eu-west-1-848439228145/Capstone/Udacity_MAILOUT_052018_TEST.csv', data_type='S3Prefix', content_type='text/csv', split_type='Line')
pipeline_transformer.wait()

2020-05-23 10:40:18,685 INFO sagemaker: Creating transform job with name: AutoML-ArvatoExpe-notebook-run-23-08-53-2020-05-23-10-40-18-507
......................Processing /opt/ml/code
  Created wheel for sagemaker-serve: filename=sagemaker_serve-1.0.0-py2.py3-none-any.whl size=8398 sha256=5df8033ada5692a949d2574bfa722d91f221315f5ce6599419db1d48a3d8e362
  Stored in directory: /tmp/pip-ephem-wheel-cache-vgcmb185/wheels/3e/0f/51/2f1df833dd0412c1bc2f5ee56baac195b5be563353d111dca6
Successfully built sagemaker-serve
Processing /opt/ml/code
  Created wheel for sagemaker-serve: filename=sagemaker_serve-1.0.0-py2.py3-none-any.whl size=8398 sha256=5df8033ada5692a949d2574bfa722d91f221315f5ce6599419db1d48a3d8e362
  Stored in directory: /tmp/pip-ephem-wheel-cache-vgcmb185/wheels/3e/0f/51/2f1df833dd0412c1bc2f5ee56baac195b5be563353d111dca6
Successfully built sagemaker-serve
[2020-05-23 10:43:49 +0000] [11] [INFO] Starting gunicorn 19.10.0
[2020-05-23 10:43:49 +0000] [11] [INFO] Listening at: unix:/tm